<a href="https://colab.research.google.com/github/burke-up/test_celery/blob/main/TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install pytube

In [2]:
%%capture
!pip install transformers[sentencepiece] accelerate

设置保存路径

In [3]:
video_save_path = "path_to_save_video"
audio_save_path = "path_to_save_audio"

下载youtube视频，并提取音频，保存到文件中

In [4]:
from pytube import YouTube

video_id = "eqOfr4AGLk8"

# 输入YouTube视频的URL
video_url = "https://www.youtube.com/watch?v=%s"%(video_id)

# 创建YouTube对象
yt = YouTube(video_url)

# 获取视频的所有流
streams = yt.streams

# 选择要下载的视频流，这里选择第一个mp4视频流
video_stream = streams.filter(file_extension='mp4').first()

# 下载视频
video_stream.download(video_save_path)

# 提取音频，选择一个音频流
audio_stream = streams.filter(only_audio=True).first()

# 下载音频
audio_stream.download(audio_save_path)

print("下载完成！")

下载完成！


查看保存的音频文件

In [5]:
!ls path_to_save_audio

'LangChain Data Loaders Tokenizers Chunking and Datasets - Data Prep 101.mp4'


以更友好的方式显示音频文件

In [6]:
from IPython.display import Audio, display
import os
files = os.listdir(audio_save_path)
audio_file = ""
if files:
  audio_file = os.path.join(audio_save_path, files[0])
  display(Audio(audio_file))
else:
  print("not found audio")



使用whisper-large-v3，将音频转换为文本，并提取时间戳

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

sample = audio_file



In [8]:
result = pipe(sample)
import json
print(json.dumps(result))

{"text": " In this video, we are going to take a look at what we need to do and what we need to consider when we are chunking text for large language models. The best way I can think of demonstrating this is to walk through an example. Now, we're going to really go with what I believe is kind of like a rule of thumb that I tend to use when I'm chunking text in order to put into a large language model. And it doesn't necessarily apply to every use case. Every use case is slightly different. But I think this is a pretty good approach, at least when we're using retrieval augmentation and large language models, which I think is where the chunking question kind of comes up most often. So let's jump straight into it. In this example what we're going to be doing is taking the langchain docs here literally every page on this website and we're going to be downloading those taking each one of these pages and I'm going to be splitting them into more reasonably sized chunks. Now how are we going t

In [21]:
import locale
locale.getdefaultlocale()
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [22]:
!pip install git+https://github.com/suno-ai/bark.git


  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-luse2ihe
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-luse2ihe
  Resolved https://github.com/suno-ai/bark.git to commit 773624d26db84278a55aacae9a16d7b25fbccab8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.8 MB/s eta 0:00:00
  Created wheel for suno-bark: filename=suno_bark-0.0.1a0-py3-none-any.whl size=2567

In [11]:
from transformers import pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-zh")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/806k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/805k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [1]:
import scipy

text_to_speech_convertor = pipeline("text-to-speech", model="suno/bark")



NameError: ignored

In [14]:
for row in result["chunks"][0:10]:
  translate_text = translator(row["text"])
  print("origin:%s"%row["text"])
  print("translate:%s"%(translate_text))
  speech = text_to_speech_convertor(translate_text)
  # scipy.io.wavfile.write("bark_out.wav", rate=speech["sampling_rate"], data=speech["audio"])
  # Audio(data=audio_data, rate=22050)
  Audio(rate=speech["sampling_rate"], data=speech["audio"])





origin: In this video, we are going to take a look at what we need to do and what we need to consider
translate:[{'translation_text': '在这个视频中,我们将审视我们需要做什么和需要考虑什么。'}]
origin: when we are chunking text for large language models. The best way I can think of demonstrating
translate:[{'translation_text': '当我们为大语言模型的文本加块时。 最好的演示方式'}]
origin: this is to walk through an example. Now, we're going to really go with what I believe is kind
translate:[{'translation_text': '举个例子。现在,我们要用我认为是善良的'}]
origin: of like a rule of thumb that I tend to use when I'm chunking text in order to put into a
translate:[{'translation_text': '就像一个大拇指规则,我倾向于使用 当我在填充文字时, 以便放入一个'}]
origin: large language model. And it doesn't necessarily apply to every use case. Every use case is
translate:[{'translation_text': '大语言模型。 它不一定适用于每个使用案例。 每个使用案例都是'}]
origin: slightly different. But I think this is a pretty good approach, at least when we're using retrieval
translate:[{'translation_text': '稍稍不同,但我认为这是一个相当不错的方法, 至少当我们使用检索时'}]
o